In [1]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
pd.set_option('display.max_columns', 200)

In [3]:
def percent_of_nan_rows(dataset:pd.DataFrame, column:str) -> float:
    return len(dataset[dataset[column].isna()]) / len(dataset) * 100

In [4]:
def PerformGridSearchCV(data:pd.DataFrame, *, feature_cols:list, label_col:list, label_col_type:str,
                        train_row_len:int = 1200, model:str, cv:int):
    __docstring__ = 'Model can be "xgb", "rf", "dt", "nb".\nLabec col can be categorical "cat" or binary "bin".'
    X_train, X_test, y_train, y_test = (data[feature_cols][:train_row_len],
                                        data[feature_cols][train_row_len:],
                                        data[label_col][:train_row_len],
                                        data[label_col][train_row_len:])
    y_train = y_train.to_numpy().ravel()
    y_test = y_test.to_numpy().ravel()
    if model == 'xgb':
        model = GradientBoostingClassifier()
        param_grid = [
    {'n_estimators':[10, 100, 200, 500],
     'learning_rate':[1e-3, 1e-2, 1e-1, 1],
     'max_depth':[1, 2, 3],
    }
    ]
    elif model == 'rf':
        model = RandomForestClassifier()
        param_grid = [
    {'n_estimators':[10, 100, 200, 500],
     'criterion':['gini', 'entropy', 'log_loss'],
     'bootstrap': [True, False],
    }
    ]
    elif model == 'dt':
        model = DecisionTreeClassifier()
        param_grid = [
    {'criterion':['gini', 'entropy'],
    'class_weight':[None, 'balanced'],
     
    }
    ]
    elif model == 'nb':
        model = Pipeline([('Normalizing',MinMaxScaler()),('MultinomialNB',MultinomialNB())])
        param_grid = [{
            'verbose':[False]
        }]
    else:
        raise ValueError(f'Model should be in ["xgb", "rf", "dt", "nb"], {model} given.')
    if label_col_type == 'cat':
        scoring = ['accuracy', 'recall_weighted', 'f1_weighted', 'precision_micro']
    elif label_col_type == 'bin':
        scoring = ['accuracy', 'recall', 'f1', 'precision']
    else:
        raise ValueError(f'Label type should be in ["cat", "bin"], {label_col_type} given.')
    cols = list(map(lambda x: 'mean_test_'+x, scoring))
    cols.extend(['mean_fit_time', 'params', 'rank_test_accuracy'])
    cv_obj = GridSearchCV(model, param_grid,
                              cv=cv,
                              scoring=scoring, verbose=0,
                              n_jobs = -1,
                              refit=False,
                              )
    cv_obj.fit(X_train, y_train)
    return pd.DataFrame(cv_obj.cv_results_)[cols].sort_values(['rank_test_accuracy'])

In [5]:
myocardial_infarction_complications = fetch_ucirepo(id=579)
X = myocardial_infarction_complications.data.features
y = myocardial_infarction_complications.data.targets

In [6]:
X = X.fillna(np.nan)

In [7]:
vars_ = pd.DataFrame(myocardial_infarction_complications.variables)

In [8]:
#Drop rows with more than 20% of values are NaN
X.drop(index = X[X.count(axis=1)<len(X.columns)*0.8].index, inplace=True)

In [9]:
#Drop columns with more than 30% of values are NaN
#Note: labels columns do not contain NaN values
#
for column in X.columns:
    if percent_of_nan_rows(X, column) > 30:
        X.drop(columns = [column], inplace=True)
        vars_.drop(index = vars_.loc[vars_['name'] == column].index, axis = 0, inplace=True)
'''        
        try:
            bin_cols.remove(column)
        except:
            pass
        try:
            cat_cols.remove(column)
        except:
            pass
        try:
            int_cols.remove(column)
        except:
            pass
        try:
            float_cols.remove(column)
        except:
            pass'''

'        \n        try:\n            bin_cols.remove(column)\n        except:\n            pass\n        try:\n            cat_cols.remove(column)\n        except:\n            pass\n        try:\n            int_cols.remove(column)\n        except:\n            pass\n        try:\n            float_cols.remove(column)\n        except:\n            pass'

In [10]:
#Get column names for columns of categorical, binary, continuous and int variable types
cat_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Categorical')]['name'].tolist()
bin_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Binary')]['name'].tolist()
int_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Integer')]['name'].tolist()
float_cols = vars_[(vars_['role'] == 'Feature') & (vars_['type'] == 'Continuous')]['name'].tolist()

In [11]:
bin_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Binary')]['name'].tolist()
cat_cols_preds = vars_[(vars_['role'] == 'Target') & (vars_['type'] == 'Categorical')]['name'].tolist()

In [12]:
#y = pd.get_dummies(data = y, columns = cat_cols_preds)

In [13]:
data = X.merge(y, left_index=True, right_index=True)

In [14]:
#Multivariate iterative imputing of missing variables
imp = IterativeImputer(max_iter=100)

In [15]:
data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,MP_TP_POST,SVT_POST,GT_POST,FIB_G_POST,ant_im,lat_im,inf_im,post_im,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,R_AB_1_n,R_AB_2_n,R_AB_3_n,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_1_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,77.0,1,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.7,0.0,138.0,NaN,NaN,8.0,16.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
1,55.0,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.5,0.0,132.0,0.38,0.18,7.8,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0
2,52.0,1,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,132.0,0.30,0.11,10.8,NaN,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
3,68.0,0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,120.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.9,0.0,146.0,0.75,0.37,NaN,NaN,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0
4,60.0,1,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.5,0.0,132.0,0.45,0.22,8.3,NaN,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1693,70.0,0,0.0,2.0,2.0,1.0,2.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,140.0,80.0,0.0,0.0,0.0,0.0,0.0,

In [16]:
#Number of missing values and number of complete rows
print(f'Missing values amount: {data.isna().sum().sum()}')
print(f'Fully entered rows amount: {len(data.dropna(axis=0, how="any"))}')

Missing values amount: 4473
Fully entered rows amount: 544


In [17]:
imp.fit(data)

IterativeImputer(max_iter=100)

In [18]:
#Applying multivariate iterative imputing of missing variables /w rounding of values (needed for categorical data)
imp_data = pd.DataFrame(np.round(imp.transform(data)), columns=data.columns)

In [19]:
imp_data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,MP_TP_POST,SVT_POST,GT_POST,FIB_G_POST,ant_im,lat_im,inf_im,post_im,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,R_AB_1_n,R_AB_2_n,R_AB_3_n,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_1_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,RAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,77.0,1.0,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,138.0,0.0,0.0,8.0,16.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,55.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,132.0,0.0,0.0,8.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,52.0,1.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,180.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,132.0,0.0,0.0,11.0,10.0,3.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,68.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,120.0,70.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,146.0,1.0,0.0,9.0,14.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,60.0,1.0,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0,90.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,132.0,0.0,0.0,8.0,16.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1555,70.0,0.0,0.0,2.0,2.0,1.0,2.0,0.0,

In [20]:
#Standardize int and float columns
for column in imp_data.columns:
    if (column in int_cols or column in float_cols):
        imp_data[column] = imp_data[column].map(
            lambda l: (l-imp_data[column].mean())/imp_data[column].std())

In [21]:
#Getting correlation matrix
corr_matrix = imp_data.corr(method='spearman')

In [22]:
identity = np.identity(n=len(corr_matrix.values))

In [23]:
where_ = np.where(corr_matrix.values-identity > 0.8)

In [24]:
where_ = np.vstack([where_[0], where_[1]])

In [25]:
#Displaying columns with significant correlation
for i in range(len(where_[0]))[::2]:
    print(corr_matrix.iloc[where_[0][i]].index[where_[0][i]], end='\t')
    print(corr_matrix.columns[where_[1][i]], end='\t')
    print(corr_matrix.iloc[where_[0][i]][where_[1][i]])

STENOK_AN	FK_STENOK	0.8450335438356769
S_AD_ORIT	D_AD_ORIT	0.8331132905991112
MP_TP_POST	ritm_ecg_p_02	0.845850627969873


In [26]:
#Miocardial rupture leads to death in 100% cases - RAZRIV is not needed as a label (repeats LET_IS_3)
imp_data.drop(columns = 'RAZRIV', inplace=True)
vars_.drop(index = vars_.loc[vars_['name'] == 'RAZRIV'].index, axis = 0, inplace=True)

In [27]:
#Paroxysms of atrial fibrillation at the time of admission ==
#ECG rhythm at the time of admission to hospital: atrial fibrillation - can be dropped to reduce dimensionality
imp_data.drop(columns = 'MP_TP_POST', inplace=True)
vars_.drop(index = vars_.loc[vars_['name'] == 'MP_TP_POST'].index, axis = 0, inplace=True)

In [28]:
imp_data

,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,ZSN_A,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,SVT_POST,GT_POST,FIB_G_POST,ant_im,lat_im,inf_im,post_im,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,TIME_B_S,R_AB_1_n,R_AB_2_n,R_AB_3_n,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_1_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,1.356955,1.0,2.0,1.0,1.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.632210,1.059206,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.116897,0.0,0.245763,-0.672061,-0.302823,-0.234990,0.227566,4.0,0.0,0.0,1.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.581310,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.567952,0.417712,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,-0.234990,-0.969282,2.0,0.0,0.0,0.0,0.0,-0.621742,-0.261379,-0.203819,1.0,-0.279280,-0.236833,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.845619,1.0,0.0,0.0,0.0,2.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.632210,1.059206,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,0.665709,-0.324825,3.0,3.0,0.0,0.0,0.0,0.659523,-0.261379,-0.203819,3.0,4.757454,5.403772,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.564028,0.0,0.0,0.0,0.0,2.0,2.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.567952,-0.865277,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,1.605905,1.141806,-0.302823,0.065243,0.043436,2.0,0.0,0.0,1.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,-0.140795,1.0,0.0,0.0,0.0,2.0,3.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.898823,0.417712,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,-0.774344,-0.672061,-0.302823,-0.234990,0.227566,9.0,0.0,0.0,0.0,0.0,-0.621742,-0.261379,-0.203819,0.0,-0.279280,-0.236833,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [29]:
#Arterial hypertension presence correlating to each other
#imp_data[['GB', 'DLIT_AG']]
#Exertional angina pectoris == Functional class (FC) of angina pectoris correlation of presence too
#imp_data[['STENOK_AN', 'FK_STENOK']]

In [30]:
imp_data = pd.get_dummies(data = imp_data, columns = cat_cols).sample(frac=1)

In [31]:
imp_data

,AGE,SEX,SIM_GIPERT,nr_11,nr_01,nr_02,nr_03,nr_04,nr_07,nr_08,np_01,np_04,np_05,np_07,np_08,np_09,np_10,endocr_01,endocr_02,endocr_03,zab_leg_01,zab_leg_02,zab_leg_03,zab_leg_04,zab_leg_06,S_AD_ORIT,D_AD_ORIT,O_L_POST,K_SH_POST,SVT_POST,GT_POST,FIB_G_POST,IM_PG_P,ritm_ecg_p_01,ritm_ecg_p_02,ritm_ecg_p_04,ritm_ecg_p_06,ritm_ecg_p_07,ritm_ecg_p_08,n_r_ecg_p_01,n_r_ecg_p_02,n_r_ecg_p_03,n_r_ecg_p_04,n_r_ecg_p_05,n_r_ecg_p_06,n_r_ecg_p_08,n_r_ecg_p_09,n_r_ecg_p_10,n_p_ecg_p_01,n_p_ecg_p_03,n_p_ecg_p_04,n_p_ecg_p_05,n_p_ecg_p_06,n_p_ecg_p_07,n_p_ecg_p_08,n_p_ecg_p_09,n_p_ecg_p_10,n_p_ecg_p_11,n_p_ecg_p_12,fibr_ter_01,fibr_ter_02,fibr_ter_03,fibr_ter_05,fibr_ter_06,fibr_ter_07,fibr_ter_08,GIPO_K,K_BLOOD,GIPER_NA,NA_BLOOD,ALT_BLOOD,AST_BLOOD,L_BLOOD,ROE,NITR_S,NA_R_1_n,NA_R_2_n,NA_R_3_n,NOT_NA_2_n,NOT_NA_3_n,LID_S_n,B_BLOK_S_n,ANT_CA_S_n,GEPAR_S_n,ASP_S_n,TIKL_S_n,TRENT_S_n,FIBR_PREDS,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANC,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS,INF_ANAM_0.0,INF_ANAM_1.0,INF_ANAM_2.0,INF_ANAM_3.0,STENOK_AN_0.0,STENOK_AN_1.0,STENOK_AN_2.0,STENOK_AN_3.0,STENOK_AN_4.0,STENOK_AN_5.0,STENOK_AN_6.0,FK_STENOK_0.0,FK_STENOK_1.0,FK_STENOK_2.0,FK_STENOK_3.0,FK_STENOK_4.0,IBS_POST_0.0,IBS_POST_1.0,IBS_POST_2.0,GB_0.0,GB_1.0,GB_2.0,GB_3.0,DLIT_AG_0.0,DLIT_AG_1.0,DLIT_AG_2.0,DLIT_AG_3.0,DLIT_AG_4.0,DLIT_AG_5.0,DLIT_AG_6.0,DLIT_AG_7.0,DLIT_AG_8.0,DLIT_AG_9.0,ZSN_A_0.0,ZSN_A_1.0,ZSN_A_2.0,ZSN_A_3.0,ZSN_A_4.0,ant_im_0.0,ant_im_1.0,ant_im_2.0,ant_im_3.0,ant_im_4.0,lat_im_0.0,lat_im_1.0,lat_im_2.0,lat_im_3.0,lat_im_4.0,inf_im_-1.0,inf_im_0.0,inf_im_1.0,inf_im_2.0,inf_im_3.0,inf_im_4.0,post_im_0.0,post_im_1.0,post_im_2.0,post_im_3.0,post_im_4.0,TIME_B_S_1.0,TIME_B_S_2.0,TIME_B_S_3.0,TIME_B_S_4.0,TIME_B_S_5.0,TIME_B_S_6.0,TIME_B_S_7.0,TIME_B_S_8.0,TIME_B_S_9.0,R_AB_1_n_0.0,R_AB_1_n_1.0,R_AB_1_n_2.0,R_AB_1_n_3.0,R_AB_2_n_0.0,R_AB_2_n_1.0,R_AB_2_n_2.0,R_AB_2_n_3.0,R_AB_3_n_0.0,R_AB_3_n_1.0,R_AB_3_n_2.0,R_AB_3_n_3.0,NOT_NA_1_n_0.0,NOT_NA_1_n_1.0,NOT_NA_1_n_2.0,NOT_NA_1_n_3.0,NOT_NA_1_n_4.0
1419,0.740234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.898823,1.700701,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.251806,0.0,-1.114379,1.141806,3.196462,-0.234990,1.976807,1.0,0.659523,-0.261379,-0.203819,-0.279280,-0.236833,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False
520,1.268852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.934646,-0.865277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.251806,0.0,0.415781,-0.672061,-0.302823,-1.435923,-0.785152,0.0,1.940787,-0.261379,-0.203819,-0.279280,-0.236833,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,False,False,True,True,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,False,False,True,False,False,False,True,False,False,False,False
1409,0.387822,1.0,0.0,0.0,0.0,0.0,0.0,1.0

In [32]:
#Features and labels for classification
features = imp_data.columns[~imp_data.columns.isin(y.columns)]

In [33]:
labels = imp_data.columns[imp_data.columns.isin(y.columns)]

In [34]:
#To avoid the need for multi-label classification, the model will be trained on one label at a time
#Starting with Letality prediction

In [35]:
labels = labels[labels.str.startswith('LET_IS')]

In [36]:
X_train, X_test, y_train, y_test = imp_data[features][:1200], imp_data[features][1200:], imp_data[labels][:1200], imp_data[labels][1200:]
#X_dataset = imp_data[features]
#y_dataset = imp_data[labels]
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

In [37]:
features

Index(['AGE', 'SEX', 'SIM_GIPERT', 'nr_11', 'nr_01', 'nr_02', 'nr_03', 'nr_04',
       'nr_07', 'nr_08',
       ...
       'R_AB_2_n_3.0', 'R_AB_3_n_0.0', 'R_AB_3_n_1.0', 'R_AB_3_n_2.0',
       'R_AB_3_n_3.0', 'NOT_NA_1_n_0.0', 'NOT_NA_1_n_1.0', 'NOT_NA_1_n_2.0',
       'NOT_NA_1_n_3.0', 'NOT_NA_1_n_4.0'],
      dtype='object', length=172)

In [38]:
labels

Index(['LET_IS'], dtype='object')

In [39]:
#The best classification algorithm will be searched via Grid search cross-validation
#It allows the testing of best parameters for any classifier
#Classifiers tested will be:
'''
xGB
Random Forests
Decision Trees
Multinomial Naive Bayes
'''
model_list = ['xgb', 'rf', 'dt', 'nb']

In [40]:
cv_ = 6

In [41]:
for model in model_list:
    res = PerformGridSearchCV(imp_data, feature_cols=features, label_col=labels, label_col_type='cat',
                             model=model, cv=cv_)
    

/home/eugen/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/eugen/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/eugen/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/eugen/anaconda3/envs/tf/lib/python3.9/site-packages/sklearn/utils/_param_validation.py:591: FutureWarning: Passing an int for a boolean parameter is deprecated in version 1.2 and won't be supported anymore in version 1.4.
  warnings.warn(
/home/eugen/anaconda

In [42]:
res

,mean_test_accuracy,mean_test_recall_weighted,mean_test_f1_weighted,mean_test_precision_micro,mean_fit_time,params,rank_test_accuracy
0,0.839167,0.839167,0.809811,0.839167,0.034782,{'verbose': 0},1


In [43]:
res.columns

Index(['mean_test_accuracy', 'mean_test_recall_weighted',
       'mean_test_f1_weighted', 'mean_test_precision_micro', 'mean_fit_time',
       'params', 'rank_test_accuracy'],
      dtype='object')